# Golden-AI Pro: Backtesting

Backtesting SMC + AI strategy on historical data.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import vectorbt as vbt
import matplotlib.pyplot as plt

from src.analysis.smc.smc_engine import SMCEngine
from src.risk.position_sizing import PositionSizer

## 1. Load Historical Data

In [ ]:
df = pd.read_csv('../data/xauusd_h1.csv', parse_dates=['time'], index_col='time')
df = df.iloc[-5000:]  # Last 5000 bars

print(f"Data range: {df.index[0]} to {df.index[-1]}")
df.head()

## 2. Generate Signals with SMC

In [ ]:
smc = SMCEngine()

signals = []
for i in range(100, len(df)):
    window = df.iloc[i-100:i]
    signal = smc.analyze(window, timeframe='H1')
    
    signals.append({
        'timestamp': df.index[i],
        'signal': signal.type.value if signal.confidence > 0.6 else 'neutral',
        'confidence': signal.confidence,
        'entry': signal.entry_price,
        'sl': signal.stop_loss,
        'tp': signal.take_profit
    })

signals_df = pd.DataFrame(signals)
signals_df.set_index('timestamp', inplace=True)
signals_df.head(10)

## 3. Vectorized Backtest

In [ ]:
# Create entries and exits
entries = signals_df['signal'] == 'buy'
exits = signals_df['signal'] == 'sell'

# Run backtest
portfolio = vbt.Portfolio.from_signals(
    close=df['close'],
    entries=entries,
    exits=exits,
    init_cash=10000,
    fees=0.001,
    slippage=0.001
)

print(portfolio.stats())

## 4. Visualize Results

In [ ]:
fig = portfolio.plot()
fig.show()

## 5. Performance Metrics

In [ ]:
metrics = {
    'Total Return': f"{portfolio.total_return():.2%}",
    'Sharpe Ratio': f"{portfolio.sharpe_ratio():.2f}",
    'Max Drawdown': f"{portfolio.max_drawdown():.2%}",
    'Win Rate': f"{portfolio.trades.win_rate():.2%}",
    'Avg Winning Trade': f"${portfolio.trades.returns.avg():.2f}",
    'Avg Losing Trade': f"${portfolio.trades.returns[portfolio.trades.returns < 0].avg():.2f}",
    'Profit Factor': f"{portfolio.trades.profit_factor():.2f}",
    'Total Trades': portfolio.trades.count()
}

for metric, value in metrics.items():
    print(f"{metric}: {value}")